<a href="https://colab.research.google.com/github/reyllama/TTIO/blob/master/04_Rescaling_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/corr3.csv"
df = pd.read_csv(path)
df = df.set_index(keys='stock_id')
df.head()

In [ ]:
dtype = torch.FloatTensor
epochs = 100
emb_dim = 32
embeddings = Variable(torch.randn(emb_dim, len(df)).type(dtype), requires_grad=True)
lr = 0.01
optimizer = optim.Adam([embeddings], lr=lr)
mean = np.abs(df).mean().mean()
coef = 1/mean
loss_track = []

for epoch in range(epochs):
  loss = 0
  for i in range(len(df)):
    v1 = embeddings[:, i].view(emb_dim, -1)
    dist = torch.norm(v1 - embeddings, dim=0).view(len(df), 1)
    corrs = torch.from_numpy(np.abs(coef*np.array(df.iloc[:, i]))-1).view(1, len(df)).type(torch.FloatTensor)
    loss += torch.mm(corrs, dist)/2
  loss_track.append(loss.item())
  if (epoch+1) % 5 == 0:
    print("{0}th epoch in process".format(epoch+1))
    print('running loss: {}'.format(loss.item()))
    print()

  optimizer.zero_grad()
  loss.backward(retain_graph=False)
  optimizer.step()

  with torch.no_grad():
    for element in embeddings:
      element.clamp_(0,1)

embeddings[:, 1]  

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,9))
plt.plot(list(range(epochs)), loss_track)
plt.show()

In [ ]:
embeddings

tensor([[0.5709, 0.5711, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.5379, 0.5357, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [0.5088, 0.5078, 1.0000,  ..., 1.0000, 0.0000, 0.0000],
        ...,
        [0.5198, 0.5197, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.5329, 0.5329, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.5397, 0.5382, 0.0000,  ..., 0.0000, 1.0000, 0.0000]],
       requires_grad=True)

## Rescaling Network

In [ ]:
data = pd.read_csv("/content/drive/My Drive/stocks.csv")
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume
0,000020,동화약품,2016-01-04,8130,8150,7920,8140,281440
1,000020,동화약품,2016-01-05,8040,8250,8000,8190,243179
2,000020,동화약품,2016-01-06,8200,8590,8110,8550,609906
3,000020,동화약품,2016-01-07,8470,8690,8190,8380,704752
4,000020,동화약품,2016-01-08,8210,8900,8130,8770,802330


In [ ]:
data['KLength'] = data['closing_price'] - data['opening_price']
data['KUpperLength'] = data['highest_price'] - data[['opening_price', 'closing_price']].max(axis=1)
data['temp'] = data['opening_price']-data['lowest_price']
data['KLowerLength'] = data[['closing_price', 'temp']].min(axis=1)
data['return'] = data['closing_price'].diff()
data = data.drop(columns=['temp'])
data['stock_id'] = data['stock_id'].astype(str)
## 첫날 빼줘야됨 (종목 바뀔때 섞여들어갔다)
data.head()

In [ ]:
def year_parser(column):
  return int(column[0:4])

def month_parser(column):
  return int(column[5:7])

data['year'] = data['date'].apply(year_parser)
data['month'] = data['date'].apply(month_parser)

data = data[(data['year']==2016)&(data['month']<=6)]
data = data[data['stock_id'].isin(df.columns)]
# data = data.fillna(0)

KL = data.pivot(index='date', columns='stock_id', values='KLength')
R = data.pivot(index='date', columns='stock_id', values='return')

KL.shape

(121, 1863)

In [ ]:
print(KL.shape, R.shape)

(121, 2018) (121, 2018)


In [ ]:
KL = KL.iloc[:-1, ]
R = R.iloc[1:, ]

In [ ]:
dtype = torch.FloatTensor
weights = Variable(torch.rand(1, 32).type(dtype), requires_grad=True)
lr = 0.005
optimizer = optim.Adam([weights], lr=lr)
epochs = 3000
indic = np.array(KL)
returns = np.array(R)

for epoch in range(epochs):
  loss = 0

  rescale_fc = torch.exp(torch.mm(weights, embeddings)) / torch.sum(torch.exp(torch.mm(weights, embeddings)))
  opt_ind = torch.from_numpy(indic) * rescale_fc # element-wise
  # print('opt_ind: ', opt_ind)
  mean_ind = torch.mean(opt_ind, axis=0)
  # print('mean_ind: ', mean_ind)
  mean_ret = torch.mean(torch.from_numpy(returns), axis=0)
  # print('mean_ret: ', mean_ret)

  vx = opt_ind - mean_ind
  vy = torch.from_numpy(returns) - mean_ret
  # print('vx: ', vx)
  # print('vy: ', vy)
  loss = -torch.abs(torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2))))
  # print('loss: ', loss)

  if (epoch+1) % 100 == 0:
    print("{0}th epoch in process".format(epoch+1))
    print('running loss: {}'.format(loss.item()))
    print()

  optimizer.zero_grad()
  loss.backward(retain_graph=False)
  optimizer.step()

  with torch.no_grad():
    for weight in weights:
      weight.clamp_(0,1)

weights

### Comparison: Raw KLength vs Rescaled KLength

In [ ]:
mean_KL = np.mean(indic, axis=0)
mean_return = np.mean(returns, axis=0)

vx = indic - mean_KL
vy = returns - mean_return

corr1 = np.abs(np.sum(vx*vy) / (np.sqrt(np.sum(vx ** 2)) * np.sqrt(np.sum(vy ** 2))))

rescale_fc = torch.exp(torch.mm(weights, embeddings)) / torch.sum(torch.exp(torch.mm(weights, embeddings)))
opt_ind = torch.from_numpy(indic) * rescale_fc # element-wise
mean_ind = torch.mean(opt_ind, axis=0)
mean_ret = torch.mean(torch.from_numpy(returns), axis=0)

wx = opt_ind - mean_ind
wy = torch.from_numpy(returns) - mean_ret
corr2 = torch.abs(torch.sum(wx * wy) / (torch.sqrt(torch.sum(wx ** 2)) * torch.sqrt(torch.sum(wy ** 2))))

In [ ]:
print("Correlation of Raw vs Recaled: {0:.4f} vs {1:.4f}".format(corr1, corr2))

Correlation of Raw vs Recaled: 0.2428 vs 0.2473


In [ ]:
!pip install ta

  Created wheel for ta: filename=ta-0.5.25-cp36-none-any.whl size=24880 sha256=570a505dd5ce4b2c21a3fd9fc98c7c94b2e2c03ca52682f343b7a5f830117ce6
  Stored in directory: /root/.cache/pip/wheels/2e/93/b7/cf649194508e53cee4145ffb949e9f26877a5a8dd12db9ed5b
Successfully built ta


In [ ]:
import ta

def bias(close=None, m=7):
  return close - close.rolling(m).mean()

soda = data.copy()
soda['EMA'] = ta.trend.ema_indicator(close=data['closing_price'])
soda['MACD'] = ta.trend.macd(close=data['closing_price'])
soda['ROC'] = ta.momentum.roc(close=data['closing_price'])
soda['BIAS'] = bias(soda['closing_price'])
soda.iloc[3:28]

In [ ]:
path = "/content/drive/My Drive/stocks_merged.csv"
merg = pd.read_csv(path)
merg.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume,industry
0,0,20,동화약품,2016-01-04,8130,8150,7920,8140,281440,의약품 제조업
1,1,20,동화약품,2016-01-05,8040,8250,8000,8190,243179,의약품 제조업
2,2,20,동화약품,2016-01-06,8200,8590,8110,8550,609906,의약품 제조업
3,3,20,동화약품,2016-01-07,8470,8690,8190,8380,704752,의약품 제조업
4,4,20,동화약품,2016-01-08,8210,8900,8130,8770,802330,의약품 제조업


In [ ]:
merg['industry'].unique().shape

(148,)

In [ ]:
merg['industry'].unique()

### Modulate

In [37]:
path = "/content/drive/My Drive/corr3.csv"
df = pd.read_csv(path)
df = df.set_index(keys='stock_id')
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,000020,000040,000050,000060,000070,000075,000080,000087,000100,000105,000120,000140,000145,000150,000155,000157,000180,000210,000215,000220,000225,000227,000230,000240,000250,000270,000300,000320,000325,000370,000390,000400,000430,000440,000480,000490,000500,000520,000540,000545,...,95190,95270,95340,95500,95570,95610,95660,95700,95720,95910,96040,96240,96300,96350,96530,96610,96630,96640,96760,96770,96775,96870,97230,97520,97780,97800,97870,97950,97955,98120,98460,98660,99190,99220,99320,99340,99350,99410,99440,99520
stock_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000020,1.000000,0.225326,0.190993,0.091289,0.278384,0.106534,0.099041,0.118218,0.412659,0.149035,0.206372,0.345200,0.239751,0.306873,0.139372,0.083818,0.432648,0.320367,0.314789,0.477852,0.390176,0.346164,0.213423,0.009110,0.410569,0.105879,0.404843,0.133121,0.297274,0.332258,0.313026,0.379187,0.141835,0.161505,0.030559,0.196879,0.188596,0.499444,-0.028242,-0.026712,...,0.209581,0.232547,0.223981,0.328538,0.087554,0.116857,0.409012,0.511532,0.203512,0.285751,0.129244,0.153266,0.126188,0.305188,0.455201,0.086679,0.397978,0.274513,0.518429,0.076621,0.031647,0.166450,0.253614,0.197545,0.058612,0.349249,0.347917,0.187737,0.226750,0.126712,0.085994,0.004352,0.375279,0.250622,0.217417,0.139307,0.094990,0.378860,0.353696,0.209319
000040,0.225326,1.000000,0.094916,-0.004544,0.059474,0.060820,0.124682,0.056387,0.209455,0.054128,-0.075235,0.119661,0.236891,-0.060359,-0.035623,-0.035829,0.317652,0.176512,0.020059,0.328834,0.247685,0.211691,0.115099,-0.014752,0.139633,-0.096755,0.314768,0.189131,0.358526,-0.004887,0.193405,0.059967,0.100306,0.149209,0.122399,0.201767,0.035985,0.317873,-0.162010,-0.180628,...,0.016408,0.144741,0.088078,0.200713,0.163985,0.314903,0.265392,0.203224,0.180150,0.190553,0.126491,0.150775,0.060059,0.246017,0.192996,0.088002,0.305419,0.062956,0.216271,0.089516,0.072886,0.211917,0.113564,0.218824,0.149922,0.178068,0.321512,0.088847,0.075557,0.054879,0.014856,0.178596,0.206972,0.239557,0.201670,0.112794,0.182726,0.236256,0.229121,0.138442
000050,0.190993,0.094916,1.000000,-0.083876,0.053535,0.275152,-0.268272,-0.078420,0.606344,0.287126,0.116990,-0.094776,-0.187100,0.077364,0.044405,0.031376,0.086244,0.099625,0.065717,0.143305,-0.029988,-0.000439,0.136625,0.036080,0.074330,0.096108,0.037376,-0.057867,-0.014226,-0.187444,0.013196,0.028301,0.104206,-0.081945,-0.027956,0.079866,0.053133,0.010611,-0.010329,0.026074,...,0.135364,-0.037521,0.082185,0.108906,0.072816,0.078859,0.056908,0.129180,-0.035751,0.013647,0.154071,-0.002654,0.084687,0.167558,0.256896,0.053647,0.098211,0.046830,0.451140,0.012075,-0.201710,-0.140452,0.082391,0.032930,0.079592,0.176558,0.064341,0.067198,0.133828,0.052540,0.082745,0.061835,0.090441,-0.011121,0.127440,-0.062998,-0.092143,0.034857,0.005743,0.041519
000060,0.091289,-0.004544,-0.083876,1.000000,0.123051,-0.029442,0.098188,0.163410,0.002685,0.071211,-0.043771,-0.051418,0.129886,0.203375,0.195282,0.178852,0.050663,0.066275,0.109603,-0.094076,-0.045850,0.051034,0.121892,0.056922,-0.004314,0.090554,0.166817,0.076133,0.061512,0.173798,0.181533,0.223820,-0.088203,0.060220,0.022132,0.076012,0.134598,0.146300,-0.086350,-0.084896,...,-0.026554,0.061883,0.024242,0.119134,-0.119198,0.044423,0.093497,0.095074,0.239250,0.063026,0.053660,-0.017570,0.022427,0.145879,0.009135,0.019071,0.059579,-0.004432,-0.074869,0.135409,0.117887,0.165979,0.108465,0.102076,-0.015843,0.125635,0.013862,0.142093,0.147167,0.080525,0.143968,-0.053646,0.110692,-0.067616,-0.014384,0.021044,0.051751,-0.029011,0.158526,-0.000352
000070,0.278384,0.059474,0.053535,0.123051,1.000000,0.130724,0.179808,0.259878,0.191164,0.066421,0.237759,0.138221,0.178195,0.072153,0.176960,0.036070,0.226709,0.106451,-0.011214,0.206232,0.106381,0.144500,0.209959,0.026213,0.118763,0.014829,0.098732,0.195508,0.053784,0.183455,0.229235,-0.043288,0.096291,0.068382,0.137581,0.039107,0.109441,0.221605,0.010337,0.026602,...,0.107730,-0.008949,0.089749,0.200

In [3]:
data = pd.read_csv("/content/drive/My Drive/stocks.csv")
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume
0,000020,동화약품,2016-01-04,8130,8150,7920,8140,281440
1,000020,동화약품,2016-01-05,8040,8250,8000,8190,243179
2,000020,동화약품,2016-01-06,8200,8590,8110,8550,609906
3,000020,동화약품,2016-01-07,8470,8690,8190,8380,704752
4,000020,동화약품,2016-01-08,8210,8900,8130,8770,802330


In [4]:
def get_embeddings(corr_df=df, n_epochs=100, dim=32, lr=0.01):
  dtype = torch.FloatTensor
  epochs = n_epochs
  emb_dim = dim
  embeddings = Variable(torch.randn(emb_dim, len(df)).type(dtype), requires_grad=True)
  lr = lr
  optimizer = optim.Adam([embeddings], lr=lr)
  mean = np.abs(df).mean().mean()
  coef = 2 # Hyper Parameter 변경
  loss_track = []

  for epoch in range(epochs):
    loss = 0
    for i in range(len(df)):
      v1 = embeddings[:, i].view(emb_dim, -1)
      dist = torch.norm(v1 - embeddings, dim=0).view(len(df), 1)
      corrs = torch.from_numpy(np.abs(coef*np.array(df.iloc[:, i]))-1).view(1, len(df)).type(torch.FloatTensor)
      loss += torch.mm(corrs, dist)/2
    loss_track.append(loss.item())
    if (epoch+1) % 10 == 0:
      print("{0}th epoch in process".format(epoch+1))
      print('running loss: {}'.format(loss.item()))
      print()

    optimizer.zero_grad()
    loss.backward(retain_graph=False)
    optimizer.step()

    with torch.no_grad():
      for element in embeddings:
        element.clamp_(0,1)

  return embeddings

In [5]:
def get_embeddings(corr_df=df, n_epochs=100, dim=32, lr=0.01):
  dtype = torch.FloatTensor
  epochs = n_epochs
  emb_dim = dim
  embeddings = Variable(torch.randn(emb_dim, len(df)).type(dtype), requires_grad=True)
  lr = lr
  optimizer = optim.Adam([embeddings], lr=lr)
  mean = np.abs(df).mean().mean()
  coef = 2 # Hyper Parameter 변경
  loss_track = []

  for epoch in range(epochs):
    loss = 0
    for i in range(len(df)):
      v1 = embeddings[:, i].view(emb_dim, -1)
      dist = torch.norm(v1 - embeddings, dim=0).view(len(df), 1)
      corrs = torch.from_numpy(np.abs(coef*np.array(df.iloc[:, i]))-1).view(1, len(df)).type(torch.FloatTensor)
      loss += torch.mm(corrs, dist)/2
    loss_track.append(loss.item())
    if (epoch+1) % 10 == 0:
      print("{0}th epoch in process".format(epoch+1))
      print('running loss: {}'.format(loss.item()))
      print()

    optimizer.zero_grad()
    loss.backward(retain_graph=False)
    optimizer.step()

    with torch.no_grad():
      for element in embeddings:
        element.clamp_(0,1)

  return embeddings

In [8]:
!pip install ta
import ta

def bias(close=None, m=7):
  return close - close.rolling(m).mean()

def preprocessing(data=data, year=2016, start_month=7, end_month = 9, corr_df = df):

  data['stock_id'] = data['stock_id'].astype(str)
  data['year'] = data['date'].apply(lambda x: int(x[0:4]))
  data['month'] = data['date'].apply(lambda x: int(x[5:7]))

  data = data[(data['year']==year)&(data['month']<=end_month)&(data['month']>=start_month)]
  data = data[data['stock_id'].isin(df.columns)]

  data['KLength'] = data['closing_price'] - data['opening_price']
  data['KUpperLength'] = data['highest_price'] - data[['opening_price', 'closing_price']].max(axis=1)
  data['temp'] = data['opening_price']-data['lowest_price']
  data['KLowerLength'] = data[['closing_price', 'temp']].min(axis=1)
  data['return'] = data['closing_price'].diff()
  data = data.drop(columns=['temp'])

  data['EMA'] = ta.trend.ema_indicator(close=data['closing_price'])
  data['MACD'] = ta.trend.macd(close=data['closing_price'])
  data['ROC'] = ta.momentum.roc(close=data['closing_price'])
  data['BIAS'] = bias(data['closing_price'])
  

  return data

  Created wheel for ta: filename=ta-0.5.25-cp36-none-any.whl size=24880 sha256=9a458e6a71f3a4004fd8931a8e6dc6b1f53f4828ea34ef0221d832d6c390b8d4
  Stored in directory: /root/.cache/pip/wheels/2e/93/b7/cf649194508e53cee4145ffb949e9f26877a5a8dd12db9ed5b
Successfully built ta


In [61]:
def weight_optimize(data=data, indicator='KLength', lr=0.005, n_epochs=3000, embeddings=None):
  KL = data.pivot(index='date', columns='stock_id', values=indicator)
  R = data.pivot(index='date', columns='stock_id', values='return')

  if indicator in ['KLength', 'KUpperLength', 'KLowerLength']:
    KL = KL.iloc[:-1, ]
    R = R.iloc[1:, ]
  elif indicator == 'EMA':
    KL = KL.iloc[11:-1, ]
    R = R.iloc[12:, ]
  elif indicator == 'MACD':
    KL = KL.iloc[25:-1, ]
    R = R.iloc[26:, ]
  elif indicator == 'ROC':
    KL = KL.iloc[12:-1, ]
    R = R.iloc[13:, ]
  elif indicator == 'BIAS':
    KL = KL.iloc[6:-1, ]
    R = R.iloc[7:, ]
  else:
    print('Unknown Indicator!')
    return None

  dtype = torch.FloatTensor
  weights = Variable(2*torch.rand(1, 32).type(dtype)-1, requires_grad=True)
  lr = lr
  optimizer = optim.Adam([weights], lr=lr)
  epochs = n_epochs
  indic = np.array(KL)
  returns = np.array(R)

  for epoch in range(epochs):
    loss = 0

    rescale_fc = torch.exp(torch.mm(weights, embeddings)) / torch.sum(torch.exp(torch.mm(weights, embeddings)))
    opt_ind = torch.from_numpy(indic) * rescale_fc # element-wise
  # print('opt_ind: ', opt_ind)
    mean_ind = torch.mean(opt_ind, axis=0)
  # print('mean_ind: ', mean_ind)
    mean_ret = torch.mean(torch.from_numpy(returns), axis=0)
  # print('mean_ret: ', mean_ret)

    vx = opt_ind - mean_ind
    vy = torch.from_numpy(returns) - mean_ret
  # print('vx: ', vx)
  # print('vy: ', vy)
    loss = -torch.abs(torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2))))
  # print('loss: ', loss)

    if (epoch+1) % 1000 == 0:
      print("{0}th epoch in process".format(epoch+1))
      # print('running loss: {:.4f}'.format(loss.item()))

    optimizer.zero_grad()
    loss.backward(retain_graph=False)
    optimizer.step()

    with torch.no_grad():
      for weight in weights:
        weight.clamp_(0,1)

  mean_KL = np.mean(indic, axis=0)
  mean_return = np.mean(returns, axis=0)

  vx = indic - mean_KL
  vy = returns - mean_return

  corr1 = np.abs(np.sum(vx*vy) / (np.sqrt(np.sum(vx ** 2)) * np.sqrt(np.sum(vy ** 2))))

  rescale_fc = torch.exp(torch.mm(weights, embeddings)) / torch.sum(torch.exp(torch.mm(weights, embeddings)))
  opt_ind = torch.from_numpy(indic) * rescale_fc # element-wise
  mean_ind = torch.mean(opt_ind, axis=0)
  mean_ret = torch.mean(torch.from_numpy(returns), axis=0)

  wx = opt_ind - mean_ind
  wy = torch.from_numpy(returns) - mean_ret
  corr2 = torch.abs(torch.sum(wx * wy) / (torch.sqrt(torch.sum(wx ** 2)) * torch.sqrt(torch.sum(wy ** 2))))

  # print("Correlation of Raw vs Recaled: {0:.4f} vs {1:.4f}".format(corr1, corr2))

  return weights

In [62]:
def test(data=None, weights=None, indicator='KLength'):
  KL = data.pivot(index='date', columns='stock_id', values=indicator)
  R = data.pivot(index='date', columns='stock_id', values='return')

  if indicator in ['KLength', 'KUpperLength', 'KLowerLength']:
    KL = KL.iloc[:-1, ]
    R = R.iloc[1:, ]
  elif indicator == 'EMA':
    KL = KL.iloc[11:-1, ]
    R = R.iloc[12:, ]
  elif indicator == 'MACD':
    KL = KL.iloc[25:-1, ]
    R = R.iloc[26:, ]
  elif indicator == 'ROC':
    KL = KL.iloc[12:-1, ]
    R = R.iloc[13:, ]
  elif indicator == 'BIAS':
    KL = KL.iloc[6:-1, ]
    R = R.iloc[7:, ]
  else:
    print('Unknown Indicator!')
    return None

  indic = np.array(KL)
  returns = np.array(R)
  mean_KL = np.mean(indic, axis=0)
  mean_return = np.mean(returns, axis=0)
  vx = indic - mean_KL
  vy = returns - mean_return
  corr1 = np.abs(np.sum(vx*vy) / (np.sqrt(np.sum(vx ** 2)) * np.sqrt(np.sum(vy ** 2))))

  rescale_fc = torch.exp(torch.mm(weights, embeddings)) / torch.sum(torch.exp(torch.mm(weights, embeddings)))
  opt_ind = torch.from_numpy(indic) * rescale_fc
  mean_ind = torch.mean(opt_ind, axis=0)

  wx = opt_ind - mean_ind
  wy = torch.from_numpy(vy)
  corr2 = torch.abs(torch.sum(wx * wy) / (torch.sqrt(torch.sum(wx ** 2)) * torch.sqrt(torch.sum(wy ** 2))))

  # print("Correlation of Raw vs Recaled: {0:.4f} vs {1:.4f}".format(corr1, corr2))
  return corr1, corr2

In [ ]:
#### 각 종목별로 Correlation 구해서 평균 내는 방식을 취하면, Correlation Coefficient가 Unit Invariant 하므로 어떤 Weight 곱해서 Rescaling해도 결과가 달라지지 않는다.

KL = test_df.pivot(index='date', columns='stock_id', values='KLength')
R = test_df.pivot(index='date', columns='stock_id', values='return')
KL = KL.iloc[:-1, ]
R = R.iloc[1:, ]
indic = np.array(KL)
returns = np.array(R)
mean_KL = np.mean(indic, axis=0)
mean_return = np.mean(returns, axis=0)
vx = indic - mean_KL
vy = returns - mean_return
corr1 = np.abs(np.sum(vx*vy, axis=0) / (np.sqrt(np.sum(vx ** 2, axis=0)+1e-10) * np.sqrt(np.sum(vy ** 2, axis=0)+1e-10)))

# print(corr1.isnan().sum())
# corr1
rescale_fc = torch.exp(torch.mm(inds, embeddings)) / torch.sum(torch.exp(torch.mm(inds, embeddings)))
opt_ind = torch.from_numpy(indic) * rescale_fc
mean_ind = torch.mean(opt_ind, axis=0)

wx = opt_ind - mean_ind
wy = torch.from_numpy(vy)
corr2 = torch.abs(torch.sum(wx * wy, axis=0) / (torch.sqrt(torch.sum(wx ** 2, axis=0)+1e-10) * torch.sqrt(torch.sum(wy ** 2, axis=0)+1e-10)))

print("Correlation of Raw vs Recaled: {0:.4f} vs {1:.4f}".format(corr1.mean(), corr2.mean()))

#### 내일 할 일
#### 여러 가지 Technical Indicator들의 선형결합으로 예측력 시험하기
#### 즉, 각 종목에 대한 각 Indicator의 Weight를 학습해서, 이를 이용해서 여러 Indicator 들을 결합, 이 값과 리턴 사이의 상관관계 따져보기

#### EX: I_i = weight_1 * I_i,1 + weight_2 * I_i,2 + weight_3 * I_i,3

Correlation of Raw vs Recaled: 0.1197 vs 0.1197


In [38]:
embeddings = get_embeddings(corr_df=df)
train_df = preprocessing(data=data, year=2016, start_month=7, end_month=12, corr_df=df)
test_df = preprocessing(data=data, year=2017, start_month=1, end_month=3, corr_df=df)

10th epoch in process
running loss: -4120160.5

20th epoch in process
running loss: -4325860.5

30th epoch in process
running loss: -4509828.0

40th epoch in process
running loss: -4664786.0

50th epoch in process
running loss: -4782411.5

60th epoch in process
running loss: -4831067.5

70th epoch in process
running loss: -4835614.0

80th epoch in process
running loss: -4835986.5

90th epoch in process
running loss: -4836029.0

100th epoch in process
running loss: -4836029.0



In [39]:
embeddings[7]

tensor([0., 0., 0.,  ..., 1., 0., 0.], grad_fn=<SelectBackward>)

In [70]:
t1, t2 = 0, 0
e = 50
for _ in range(e):
  inds = weight_optimize(indicator='EMA', data=train_df, embeddings=embeddings, n_epochs=300)
  x, y = test(data=test_df, weights=inds, indicator='EMA')
  t1 += x
  t2 += y
print()
print("Average Corr: {0:.4f} vs {1:.4f}".format(t1/e, t2/e))


Average Corr: 0.1671 vs 0.1507


In [71]:
t1, t2 = 0, 0
e = 50
for _ in range(e):
  inds = weight_optimize(indicator='MACD', data=train_df, embeddings=embeddings, n_epochs=300)
  x, y = test(data=test_df, weights=inds, indicator='MACD')
  t1 += x
  t2 += y
print()
print("Average Corr: {0:.4f} vs {1:.4f}".format(t1/e, t2/e))


Average Corr: 0.0475 vs 0.0292


In [65]:
t1, t2 = 0, 0
e = 50
for _ in range(e):
  inds = weight_optimize(indicator='KLength', data=train_df, embeddings=embeddings, n_epochs=80)
  x, y = test(data=test_df, weights=inds, indicator='KLength')
  t1 += x
  t2 += y
print()
print("Average Corr: {0:.4f} vs {1:.4f}".format(t1/e, t2/e))


Average Corr: 0.0600 vs 0.0200


In [72]:
t1, t2 = 0, 0
e = 50
for _ in range(e):
  inds = weight_optimize(indicator='KUpperLength', data=train_df, embeddings=embeddings, n_epochs=1000)
  x, y = test(data=test_df, weights=inds, indicator='KUpperLength')
  t1 += x
  t2 += y
print()
print("Average Corr: {0:.4f} vs {1:.4f}".format(t1/e, t2/e))

1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in process
1000th epoch in 

In [73]:
t1, t2 = 0, 0
e = 50
for _ in range(e):
  inds = weight_optimize(indicator='KLowerLength', data=train_df, embeddings=embeddings, n_epochs=300)
  x, y = test(data=test_df, weights=inds, indicator='KLowerLength')
  t1 += x
  t2 += y
print()
print("Average Corr: {0:.4f} vs {1:.4f}".format(t1/e, t2/e))


Average Corr: 0.0619 vs 0.0181


In [74]:
t1, t2 = 0, 0
e = 50
for _ in range(e):
  inds = weight_optimize(indicator='ROC', data=train_df, embeddings=embeddings, n_epochs=300)
  x, y = test(data=test_df, weights=inds, indicator='ROC')
  t1 += x
  t2 += y
print()
print("Average Corr: {0:.4f} vs {1:.4f}".format(t1/e, t2/e))


Average Corr: 0.0205 vs 0.0391


In [75]:
t1, t2 = 0, 0
e = 50
for _ in range(e):
  inds = weight_optimize(indicator='BIAS', data=train_df, embeddings=embeddings, n_epochs=300)
  x, y = test(data=test_df, weights=inds, indicator='BIAS')
  t1 += x
  t2 += y
print()
print("Average Corr: {0:.4f} vs {1:.4f}".format(t1/e, t2/e))


Average Corr: 0.0777 vs 0.0567


In [26]:
(0.1663+0.0376+0.0216+0.0439+0.0374+0.0224+0.0431)/7, (0.1417+0.0388+0.0129+0.0419+0.0351+0.0441+0.0922)/7

(0.05318571428571428, 0.0581)